In [1]:
import pandas as pd

In [2]:
site = pd.read_csv('site_info.csv')
energy = pd.read_csv('energy.csv')

In [3]:
dangjin_fcst = pd.read_csv('dangjin_fcst_final.csv')
ulsan_fcst = pd.read_csv('ulsan_fcst_final.csv')

In [4]:
def convert_24_to_00(dt):
    dt = [i for i in dt]
    dt_join = ''.join(dt)
    if dt_join[11:13] == '24':
        dt[11:13] = ['0','0']
        
        if not(dt_join[5:10] in ['01-31', '02-29', '03-31', '04-30', '05-31', '06-30',
                               '07-31', '08-31' , '09-30', '10-31', '11-30' , '12-31']):
            if len(str(int(dt_join[8:10]) + 1)) == 2:
                dt[8:10] = [i for i in str(int(dt_join[8:10]) + 1)]
            else:
                dt[8:10] = [i for i in '0' + str(int(dt_join[8:10]) + 1)]
        else:
            dt[8:10]=['0','1']
            
            if dt_join[5:7] != '12':
                if len(str(int(dt_join[5:7])+1))==2:
                    dt[5:7] = [i for i in str(int(dt_join[5:7]) + 1)]
                else:
                    dt[5:7] = [i for i in '0' + str(int(dt_join[5:7])+1)]
            else:
                dt[5:7] = ['0', '1']
                dt[0:4] = [i for i in str(int(dt_join[0:4])+1)]
    if int(dt_join[0:4]) % 4 != 0 and dt_join[5:10] == '02-28':
        dt[5:7] = ['0','3']
        dt[8:10] = ['0', '1']
        
    return ''.join(dt)

In [5]:
energy['time']=energy['time'].apply(lambda x : convert_24_to_00(x))

In [6]:
ulsan_fcst = ulsan_fcst.sort_values(by=['target time'])
dangjin_fcst = dangjin_fcst.sort_values(by=['target time'])

In [7]:
ulsan_fcst = ulsan_fcst.reset_index(drop=True)

In [8]:
import numpy as np
import scipy.interpolate as ip
from scipy.interpolate import splrep, splev
import matplotlib.pyplot as plt


In [9]:
ulsan_fcst_train = ulsan_fcst.iloc[:8761,:]
dangjin_fcst_train = dangjin_fcst.iloc[:8761,:]

In [10]:
ulsan_fcst_train

,target time,Temperature,Humidity,Cloud,Wind_x,Wind_y
0,2018-03-02 00:00:00,0.0,40.0,2.0,1.010290,-0.818117
1,2018-03-02 03:00:00,-1.0,45.0,2.0,0.872657,-1.574315
2,2018-03-02 06:00:00,-1.0,45.0,1.0,1.006547,-1.492268
3,2018-03-02 09:00:00,0.0,35.0,1.0,1.225671,-1.028460
4,2018-03-02 12:00:00,6.0,25.0,1.0,1.160653,-0.782870
...,...,...,...,...,...,...
8756,2021-02-28 18:00:00,14.0,70.0,4.0,-2.874270,1.161280
8757,2021-02-28 21:00:00,14.0,75.0,4.0,-3.864814,1.884996
8758,2021-03-01 00:00:00,13.0,80.0,4.0,-3.176148,0.389982
8759,2021-03-01 06:00:00,13.0,85.0,4.0,-2.813858,0.701573


In [11]:
ulsan_temp = ulsan_fcst_train['Temperature']
ulsan_hum = ulsan_fcst_train['Humidity']
ulsan_cloud = ulsan_fcst_train['Cloud'] -1 / 3
ulsan_wind_x = ulsan_fcst_train['Wind_x']
ulsan_wind_y = ulsan_fcst_train['Wind_y']

In [12]:
dangjin_temp = dangjin_fcst_train['Temperature']
dangjin_hum = dangjin_fcst_train['Humidity']
dangjin_cloud = dangjin_fcst_train['Cloud'] -1 / 3
dangjin_wind_x = dangjin_fcst_train['Wind_x']
dangjin_wind_y = dangjin_fcst_train['Wind_y']

In [13]:
x0 = np.linspace(0,26280,8761)
spl11 = splrep(x0, ulsan_temp)
spl12 = splrep(x0, ulsan_hum)
spl13 = splrep(x0, ulsan_cloud, k = 1)
spl14 = splrep(x0, ulsan_wind_x)
spl15 = splrep(x0, ulsan_wind_y)
spl21 = splrep(x0, dangjin_temp)
spl22 = splrep(x0, dangjin_hum)
spl23 = splrep(x0, dangjin_cloud, k = 1)
spl24 = splrep(x0, dangjin_wind_x)
spl25 = splrep(x0, dangjin_wind_y)

In [14]:
x1 = np.linspace(0,26280,26281)
y11 = splev(x1, spl11)
y12 = splev(x1, spl12)
y13 = splev(x1, spl13)
y14 = splev(x1, spl14)
y15 = splev(x1, spl15)
y21 = splev(x1, spl21)
y22 = splev(x1, spl22)
y23 = splev(x1, spl23)
y24 = splev(x1, spl24)
y25 = splev(x1, spl25)

In [15]:
train_date = dangjin_fcst_train['target time']

In [16]:
temp = list()
for i in train_date:
    if i[-8:] == '00:00:00':
        temp.append(i[:-8] + "01:00:00")
        temp.append(i[:-8] + "02:00:00")
    
    if i[-8:] == '03:00:00':
        temp.append(i[:-8] + "04:00:00")
        temp.append(i[:-8] + "05:00:00")
        
    if i[-8:] == '06:00:00':
        temp.append(i[:-8] + "07:00:00")
        temp.append(i[:-8] + "08:00:00")
        
    if i[-8:] == '09:00:00':
        temp.append(i[:-8] + "10:00:00")
        temp.append(i[:-8] + "11:00:00")
        
    if i[-8:] == '12:00:00':
        temp.append(i[:-8] + "13:00:00")
        temp.append(i[:-8] + "14:00:00")
        
    if i[-8:] == '15:00:00':
        temp.append(i[:-8] + "16:00:00")
        temp.append(i[:-8] + "17:00:00")
        
    if i[-8:] == '18:00:00':
        temp.append(i[:-8] + "19:00:00")
        temp.append(i[:-8] + "20:00:00")
        
    if i[-8:] == '21:00:00':
        temp.append(i[:-8] + "22:00:00")
        temp.append(i[:-8] + "23:00:00")

In [17]:
date_final = train_date.to_list() + temp

In [18]:
date_final = sorted(date_final)

In [20]:
train_ulsan = pd.DataFrame(date_final)
train_dangjin = pd.DataFrame(date_final)

In [21]:
train_ulsan = train_ulsan[:26281]
train_dangjin = train_dangjin[:26281]

In [22]:
train_ulsan.columns = ['date']
train_dangjin.columns = ['date']

In [24]:
train_ulsan['Temperatrue'] = y11
train_ulsan['Humidity'] = y12
train_ulsan['Cloud'] = y13
train_ulsan['Wind_x'] = y14
train_ulsan['Wind_y'] = y15

train_dangjin['Temperatrue'] = y21
train_dangjin['Humidity'] = y22
train_dangjin['Cloud'] = y23
train_dangjin['Wind_x'] = y24
train_dangjin['Wind_y'] = y25

In [28]:
train_ulsan
train_ulsan['date'] = pd.to_datetime(train_ulsan['date'])

In [29]:
train_dangjin
train_dangjin['date'] = pd.to_datetime(train_dangjin['date'])

In [30]:
def sun_system_1(date, lat, longt):
    from datetime import datetime, timedelta
    import math
    import numpy as np
    pi = math.pi
    lat = lat / 180 * pi
    temp_long = 135 - longt
    
    N = date.apply(lambda x : int((x - datetime(int(x.year), 1, 1, 0, 0)).days) + 1)
    h = date.apply(lambda x : ((int(x.hour) - 12) * 15 - temp_long) / 180 * pi)
    delta = -23.44 * np.cos(2*pi/365*(N+10)) / 180 * pi
    sin_alpha = math.sin(lat) * np.sin(delta) + math.cos(lat) * np.cos(delta) * np.cos(h)
    sin_theta = np.sqrt(1 - sin_alpha**2)
    sin_phi = -1 * np.sin(h) * np.cos(delta) / sin_theta
    cos_phi = (np.sin(delta) * math.cos(lat) - np.cos(h) * np.cos(delta) * math.sin(lat)) / sin_theta
       
        
    df = pd.concat([sin_alpha, sin_phi, cos_phi], axis=1)
    df.columns = ['sin(elev ang)', 'sin(azimuth ang)', 'cos(azimuth ang)']
    return(df)

In [31]:
tri1 = sun_system_1(train_dangjin['date'],site['Latitude'][0],site['Longitude'][0])
tri2 = sun_system_1(train_dangjin['date'],site['Latitude'][1],site['Longitude'][1])
tri3 = sun_system_1(train_dangjin['date'],site['Latitude'][2],site['Longitude'][2])
tri4 = sun_system_1(train_ulsan['date'],site['Latitude'][3],site['Longitude'][3])

In [32]:
train_dangjin_floating = pd.concat([train_dangjin, tri1], axis = 1)
train_dangjin_warehouse = pd.concat([train_dangjin, tri2], axis = 1)
train_dangjin_ = pd.concat([train_dangjin, tri3], axis = 1)
train_ulsan = pd.concat([train_ulsan, tri4], axis = 1)

In [33]:
train_dangjin_floating = train_dangjin_floating[1:]
train_dangjin_warehouse = train_dangjin_warehouse[1:]
train_dangjin_ = train_dangjin_[1:]
train_ulsan = train_ulsan[1:]

In [34]:
train_dangjin_

,date,Temperatrue,Humidity,Cloud,Wind_x,Wind_y,sin(elev ang),sin(azimuth ang),cos(azimuth ang)
1,2018-03-02 01:00:00,-2.104535,50.965453,0.666667,4.815127,-2.354512,-0.869148,0.227027,0.973889
2,2018-03-02 02:00:00,-2.483628,52.772362,0.666667,4.658834,-2.144641,-0.819200,0.633111,0.774061
3,2018-03-02 03:00:00,-3.000000,55.000000,0.666667,4.667902,-1.791840,-0.719143,0.847892,0.530169
4,2018-03-02 04:00:00,-3.516372,57.227638,0.666667,4.680548,-1.358489,-0.575798,0.948016,0.318222
5,2018-03-02 05:00:00,-3.895465,59.034547,0.666667,4.534988,-0.906971,-0.398932,0.990416,0.138119
...,...,...,...,...,...,...,...,...,...
26276,2021-03-01 02:00:00,5.826738,80.264023,3.666667,2.411359,1.943115,-0.822436,0.637681,0.770301
26277,2021-03-01 06:00:00,6.000000,90.000000,3.666667,3.064178,2.571150,-0.204422,0.999580,-0.028986
26278,2021-03-01 07:00:00,6.395484,96.958199,3.666667,3.084041,3.171088,0.001667,0.982934,-0.183960
26279,2021-03-01 08:00:00,7.049911,98.003304,3.666667,2.042855,3.663807,0.201655,0.939713,-0.341965


In [29]:
energy = energy[24:]
energy = energy.reset_index(drop=True)

In [30]:
train_dangjin_floating = train_dangjin_floating.reset_index(drop=True)
train_dangjin_warehouse = train_dangjin_warehouse.reset_index(drop=True)
train_dangjin_ = train_dangjin_.reset_index(drop=True)
train_ulsan = train_ulsan.reset_index(drop=True)

In [31]:
train_dangjin_floating = train_dangjin_floating.iloc[:26279,:]
train_dangjin_warehouse = train_dangjin_warehouse.iloc[:26279,:]
train_dangjin_ = train_dangjin_.iloc[:26279,:]
train_ulsan = train_ulsan.iloc[:26279,:]

In [32]:
energy[energy['time']!=train_dangjin_['date'][:25608]]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-02 1:00:00,0.0,0.0,0,0
1,2018-03-02 2:00:00,0.0,0.0,0,0
2,2018-03-02 3:00:00,0.0,0.0,0,0
3,2018-03-02 4:00:00,0.0,0.0,0,0
4,2018-03-02 5:00:00,0.0,0.0,0,0
...,...,...,...,...,...
25603,2021-01-31 20:00:00,0.0,0.0,0,0
25604,2021-01-31 21:00:00,0.0,0.0,0,0
25605,2021-01-31 22:00:00,0.0,0.0,0,0
25606,2021-01-31 23:00:00,0.0,0.0,0,0


In [33]:
energy['time']=pd.to_datetime(energy['time'])
train_dangjin_['date'] = pd.to_datetime(train_dangjin_['date'])

In [34]:
energy['time'][energy['time']!=train_dangjin_['date'][:25608]]

8712   2019-03-01 01:00:00
8713   2019-03-01 02:00:00
8714   2019-03-01 03:00:00
8715   2019-03-01 04:00:00
8716   2019-03-01 05:00:00
8717   2019-03-01 06:00:00
8718   2019-03-01 07:00:00
8719   2019-03-01 08:00:00
8720   2019-03-01 09:00:00
8721   2019-03-01 10:00:00
8722   2019-03-01 11:00:00
8723   2019-03-01 12:00:00
8724   2019-03-01 13:00:00
8725   2019-03-01 14:00:00
8726   2019-03-01 15:00:00
8727   2019-03-01 16:00:00
8728   2019-03-01 17:00:00
8729   2019-03-01 18:00:00
8730   2019-03-01 19:00:00
8731   2019-03-01 20:00:00
8732   2019-03-01 21:00:00
8733   2019-03-01 22:00:00
8734   2019-03-01 23:00:00
Name: time, dtype: datetime64[ns]

In [35]:
energy['time'] = pd.to_datetime(energy['time'])

In [36]:
test_dangjin_floating = train_dangjin_floating.iloc[25608:]
test_dangjin_warehouse = train_dangjin_warehouse.iloc[25608:]
test_dangjin_ = train_dangjin_.iloc[25608:]
test_ulsan = train_ulsan.iloc[25608:]

train_dangjin_floating = train_dangjin_floating.iloc[:25608]
train_dangjin_warehouse = train_dangjin_warehouse.iloc[:25608]
train_dangjin_ = train_dangjin_.iloc[:25608]
train_ulsan = train_ulsan.iloc[:25608]

In [37]:
train_dangjin_floating['energy'] = energy['dangjin_floating']
train_dangjin_warehouse['energy'] = energy['dangjin_warehouse']
train_dangjin_['energy'] = energy['dangjin']
train_ulsan['energy'] = energy['ulsan']

In [38]:
test_dangjin_floating.to_csv('test_dangjin_floating_0526.csv', index=False)
test_dangjin_warehouse.to_csv('test_dangjin_warehouse_0526.csv', index=False)
test_dangjin_.to_csv('test_dangjin_0526.csv', index=False)
test_ulsan.to_csv('test_ulsan_0526.csv', index=False)

train_dangjin_floating.to_csv('train_dangjin_floating_0526.csv', index=False)
train_dangjin_warehouse.to_csv('train_dangjin_warehouse_0526.csv', index=False)
train_dangjin_.to_csv('train_dangjin_0526.csv', index=False)
train_ulsan.to_csv('train_ulsan_0526.csv', index=False)